In [1]:
import os
from os import listdir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from pathlib import Path

import glob
import random
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset  # Ensure Dataset is imported
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim

import cv2
from matplotlib.image import imread

import tensorflow as tf
#from keras.utils.np_utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import glob
import PIL
import random

random.seed(100)

ModuleNotFoundError: No module named 'plotly'

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

# Set parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 25
DATA_PATH = 'C:/Users/moham/OneDrive - Virginia Tech/Summer-Fall 2024/CS 5805/ML Project/CODE/All Images'  # Update this to the dataset path

# Load and preprocess images
def load_images(data_path):
    images, labels = [], []
    classes = {'benign': 0, 'malignant': 1}  # Map class names to integers

    for label in os.listdir(data_path):
        class_path = os.path.join(data_path, label)
        if os.path.isdir(class_path):
            for img_name in tqdm(os.listdir(class_path), desc=f"Loading {label} images"):
                img_path = os.path.join(class_path, img_name)
                try:
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    images.append(img)
                    labels.append(classes[label])
                except Exception as e:
                    print(f"Error reading {img_path}: {e}")

    return np.array(images), np.array(labels)

# Load images
images, labels = load_images(DATA_PATH)
images = images / 255.0  # Normalize pixel values to [0, 1]

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(rotation_range=15,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=True)
datagen.fit(X_train)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                    validation_data=(X_val, y_val),
                    epochs=EPOCHS)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}")

# Save the model
model.save("breakhis_cnn_model.h5")
print("Model saved as breakhis_cnn_model.h5")

# Visualize training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Model Loss')

plt.show()


In [ ]:
#Not used
import os
import pandas as pd

# Define the path to the folder
folder_path = destination_dir

# Create dictionaries to store file information
patient_ID = {}
patient_Diag = {}

# Loop through each file in the folder to extract IDs and Diagnoses
for i, filename in enumerate(os.listdir(folder_path), start=1):
    if os.path.isfile(os.path.join(folder_path, filename)):
        # Split filename and extension
        name, ext = os.path.splitext(filename)

        # Extract Patient ID (e.g., characters 2 to 6)
        trimmed_name = name[4:]  # Adjust slicing as needed
        truncated_id = trimmed_name[:1]
        patient_Diag[i] = truncated_id

        # Extract Patient Diagnosis (e.g., characters 6 onward)
        diag_name = name[8:]
        truncated_diag = diag_name[:16]
        patient_ID[i] = truncated_diag

# Convert dictionaries to lists
id_list = list(patient_ID.values())
diag_list = list(patient_Diag.values())

# Assuming `features_list` is already created from feature extraction
df_features = pd.DataFrame(features_list, columns=columns)

# Add patient_ID and patient_Diag as columns to the DataFrame
df_features['patient_ID'] = id_list
df_features['patient_Diag'] = diag_list

# Save to CSV or print the DataFrame
df_features.to_csv("BreaKHis_full_extracted_features.csv", index=False)
print(df_features)


In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Bagging Classifier
class BaggingClassifier:
    def __init__(self, base_estimator, n_estimators=10, random_state=None):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.models = []
        self.samples = []

    def fit(self, X, y):
        np.random.seed(self.random_state)
        self.models = []
        self.samples = []

        for _ in range(self.n_estimators):
            # Bootstrap sampling
            indices = np.random.choice(range(len(X)), size=len(X), replace=True)
            X_sample, y_sample = X[indices], y[indices]
            self.samples.append(indices)

            # Train a new model
            model = self.base_estimator
            model.fit(X_sample, y_sample)
            self.models.append(model)

    def predict(self, X):
        # Aggregate predictions from all models (majority vote for classification)
        predictions = np.array([model.predict(X) for model in self.models])
        # Majority vote
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)
        return final_predictions

# Load a dataset
data = df_features
X, y = data.data, data.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the Bagging Classifier with a DecisionTreeClassifier as the base estimator
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42), n_estimators=10, random_state=42)

# Train the Bagging Classifier
bagging_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_clf.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Bagging Classifier: {accuracy:.2f}")


In [ ]:
# Define the path to the folder
folder_path = destination_dir

# Create a dictionary to store file names
patient_ID = {}

# Loop through each file in the folder
for i, filename in enumerate(os.listdir(folder_path), start=1):
    # Only add files (not directories) to the dictionary
    if os.path.isfile(os.path.join(folder_path, filename)):
        name, ext = os.path.splitext(filename)
        trimmed_name = name[2:]
        truncated_name = trimmed_name[:5]
        patient_ID[i] = truncated_name
resolution= {}

# Loop through each file in the folder
for i, filename in enumerate(os.listdir(folder_path), start=1):
    # Only add files (not directories) to the dictionary
    if os.path.isfile(os.path.join(folder_path, filename)):
        name, ext = os.path.splitext(filename)
        trimmed_name = name[19:]
        truncated_name = trimmed_name[:22]
        resolution[i] = truncated_name
# Output the dictionary

# Create a dictionary to store file names
patient_Diag = {}

# Loop through each file in the folder
for i, filename in enumerate(os.listdir(folder_path), start=1):
    # Only add files (not directories) to the dictionary
    if os.path.isfile(os.path.join(folder_path, filename)):
        name, ext = os.path.splitext(filename)
        trimmed_name = name[6:]
        truncated_name = trimmed_name[:16]
        patient_Diag[i] = truncated_name

# Output the dictionary
print(resolution)


In [ ]:
for label in os.listdir(DATA_PATH):
    class_path = os.path.join(DATA_PATH, label)
    print(os.path.isdir(class_path))
    # os.path.isdir(class_path)


In [ ]:
#To separate names
for im in list_of_path:
    print(im.split('.')[0].split('-')[0].split('_')[1],im.split('.')[0].split('-')[1]+'-'+im.split('.')[0].split('-')[2],im.split('.')[0].split('-')[3],im.split('.')[0].split('-')[0].split('_')[2])

In [ ]:
file_path = 'BreaKHis_full_extracted_features.csv'
data = pd.read_csv(file_path)
data

In [ ]:
destination_dir = 'C:/Users/moham/OneDrive - Virginia Tech/Summer-Fall 2024/CS 5805/ML Project/CODE/All Images'
list_of_path=[]
i=0
for root, dirs, files in os.walk(destination_dir):
    for file in files:
        list_of_path.append(file)
        i+=1
list_of_path

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

# Load dataset
file_path = 'BreaKHis_full_extracted_features.csv'
data = pd.read_csv(file_path)
#Only 100 resolution
# data=data.loc[data['resolution']==100]

# Preprocessing
# Drop non-numerical 'patient_ID' and handle missing values
X = data.drop(columns=['file_name','resolution','patient_Diag', 'patient_ID','tumor_type'])
X = X.fillna(X.mean())

# Encode target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['patient_Diag'])

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Bagging Classifier with Decision Tree
base_model = DecisionTreeClassifier(random_state=42)
bagging_clf = BaggingClassifier(estimator=base_model, random_state=42)  # Corrected parameter name

# Hyperparameter Tuning
param_grid = {
    'n_estimators': [10, 50, 100,200,500],  # Number of trees
    'estimator__max_depth': [5, 10, 15,20],  # Depth of decision trees
    'estimator__min_samples_split': [2, 5, 10,20],  # Minimum samples for splitting
}

grid_search = GridSearchCV(estimator=bagging_clf, param_grid=param_grid, cv=10, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best Model
best_model = grid_search.best_estimator_

# Test Set Evaluation
y_pred = best_model.predict(X_test)

# Performance Metrics
train_accuracy = accuracy_score(X_train, best_model.predict(X_train))
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

# Output results
print("Best Hyperparameters:", grid_search.best_params_)
print(f"Test Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report_str)